# King County Housing Price Prediction

## Overview

This happens in King County Seattle not BK

, often referred to as Brooklyn, is one of the most popular neighborhood in United States. Therefore, evaluating and predicting housing prices accurately provides. immense value for all participants of real estate transaction in this neighborhood. This project seeks to analyze and model the given Kings County real estate data to predict future sale prices of different properties. 

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as ans
import os
#%matplotlib.inline

In [7]:
!dir data

 Volume in drive D is Data
 Volume Serial Number is 8601-2CF3

 Directory of D:\Kings_County_Housing_Price_Prediction\data

10/16/2020  03:58 PM    <DIR>          .
10/16/2020  03:58 PM    <DIR>          ..
10/16/2020  03:58 PM           486,579 kc_house_data_test_features.csv
10/16/2020  03:58 PM         2,118,903 kc_house_data_train.csv
               2 File(s)      2,605,482 bytes
               2 Dir(s)  933,080,879,104 bytes free


In [12]:
kc_df = pd.read_csv("data/kc_house_data_test_features.csv", index_col=0)

In [13]:
kc_df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916
